<a href="https://colab.research.google.com/github/datasistah/ml_system_design_class_notebooks/blob/main/FAISS_Expanded_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

import requests
import os

data_url = "https://raw.githubusercontent.com/jamescalam/data/main/sentence_embeddings_15K/"
s=[]
# create data directory to store data
if not os.path.exists('./data'):
    os.mkdir('./data')

# download the numpy binary files (dense vectors)
for i in range(57):
    if i < 10:
        i = '0' + str(i)
    res = requests.get(data_url+f"embeddings_{i}.npy")

    with open(f'./data/embeddings_{i}.npy', 'wb') as fp:
        for chunk in res:
            fp.write(chunk)
           
    print('.', end='')

# and download the respective text file
res = requests.get(f"{data_url}sentences.txt")
with open(f"./data/sentences.txt", 'wb') as fp:
    for chunk in res:
        fp.write(chunk)

.........................................................

In [10]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [11]:
import pandas as pd

import requests
from io import StringIO
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [12]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [14]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [15]:
# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    
    smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # data=data.append(smalldata,ignore_index=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(smalldata[1].tolist())
    sentences.extend(smalldata[2].tolist())

<ipython-input-15-4e63cc361411>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 191: expected 3 fields, saw 4
Skipping line 206: expected 3 fields, saw 4
Skipping line 295: expected 3 fields, saw 4
Skipping line 695: expected 3 fields, saw 4
Skipping line 699: expected 3 fields, saw 4

<ipython-input-15-4e63cc361411>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 104: expected 3 fields, saw 4
Skipping line 181: expected 3 fields, saw 4
Skipping line 317: expected 3 fields, saw 4
Skipping line 412: expected 3 fields, saw 5
Skipping line 508: expected 3 fields, saw 4

<ipython-input-15-4e63cc36

In [16]:
smalldata.head()

,0,1,2
0,NaN,Small dog chews on a big stick.,a dog shews on a big stick.
1,NaN,A tennis player hitting the ball.,Two boys splashing in the surf.
2,NaN,a lone snowboarder in the middle of a snowy gust,A snowboarder is throwing up snow as he rides ...
3,4.4,A pair of dogs playing with a purple ball.,Two dogs play with purple football.
4,0.6,a bird lands in the water.,a boat floats in the water.


In [17]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [18]:
len(sentences)

14504

In [19]:
sentences[0]

'Griffith, 77, grew up in Mount Airy and now lives in Manteo.'

In [22]:
import numpy as np
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00


In [2]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sentence_transformers-2.2.2-py3-none-any.whl


In [1]:
# !git clone https://github.com/jamescalam/data.git data-embeddings

In [2]:
# path = '/content/data-embeddings/sentence_embeddings_15K/'
# sentence_embeddings = []
# for i in range(0,57):
#     # if i < 10:
#     #     i = '0' + str(i)
#     res = path+f"embeddings_{i}.npy"
#     print(res)
#     sm = np.load(res)
#     sentence_embeddings.append(sm)

In [3]:
# arr = np.concatenate(sentence_embeddings)

In [4]:
# arr.shape

In [5]:
# sentences = sentences[:100]

Import Model

In [6]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

Use model to encode data (embeddings)

In [20]:
%%time
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

CPU times: user 17.6 s, sys: 1.11 s, total: 18.7 s
Wall time: 22.7 s


(14504, 768)

In [ ]:
# with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
#     np.save(fp, sentence_embeddings[0:256])

In [ ]:
# # saving data
# split = 256
# file_count = 0
# for i in range(0, sentence_embeddings.shape[0], split):
#     end = i + split
#     if end > sentence_embeddings.shape[0] + 1:
#         end = sentence_embeddings.shape[0] + 1
#     file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
#     with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
#         np.save(fp, sentence_embeddings[i:end, :])
#     print(f"embeddings_{file_count}.npy | {i} -> {end}")
#     file_count = int(file_count) + 1

Use faiss to index the embeddings

In [21]:
import faiss
# sentence_embeddings = arr

In [22]:
d = sentence_embeddings.shape[1]
d

768

In [23]:
sentences[1]

'At first blush, then, the creators of the registry would seem to have drawn the line between exempted and banned calls in the right place.'

In [44]:
sentence_embeddings[0].shape

(768,)

In [31]:
#sentence_embeddings[0]

In [25]:
index = faiss.IndexFlatL2(d) # build the index, d=size of vectors 


In [26]:
index.is_trained

True

In [27]:
index.add(sentence_embeddings) #add vectors to the index

In [28]:
index.ntotal

14504

In [38]:
sentences[:5]

['Griffith, 77, grew up in Mount Airy and now lives in Manteo.',
 'At first blush, then, the creators of the registry would seem to have drawn the line between exempted and banned calls in the right place.',
 'A child swinging on a playground play set.',
 'A paper plate is being cut by a man',
 'Two young girls with dyed hair talk to two boys in black.']

In [33]:
k = 4
xq = model.encode(["Dog is running in the bushes"])

In [39]:
%%time
D, I = index.search(xq, k)  # search, k top 4 similar matches
print(I)

[[9601 6493 1379 7701]]
CPU times: user 18 ms, sys: 0 ns, total: 18 ms
Wall time: 20.5 ms


Matrix D is the matrix of squared distances. It has the same shape as I and indicates for each result vector at the query’s squared Euclidean distance.



In [35]:
D

array([[38.423836, 51.56021 , 53.2208  , 53.87866 ]], dtype=float32)

In [36]:
I

array([[9601, 6493, 1379, 7701]])

In [42]:
I[0] #the index of the most similar vector is first, then the second most similar and so on

array([9601, 6493, 1379, 7701])

In [45]:
[f'{i}: {sentences[i]}' for i in I[0]]

['9601: Two dogs run through the brush.',
 '6493: Two dogs are running on the grass in front of the trees.',
 '1379: two dogs running in grass',
 '7701: A dog is running along the grass.']

Quantizer speeds up the search

In [46]:
nlist = 100
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [47]:
index.is_trained

False

In [48]:
index.train(sentence_embeddings)
index.is_trained

True

In [49]:
index.add(sentence_embeddings)
index.ntotal

14504

In [50]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[9601 6493 1379 7701]]
CPU times: user 2.55 ms, sys: 0 ns, total: 2.55 ms
Wall time: 2.48 ms


In [51]:
index.nprobe = 10

In [52]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[9601 6493 1379 7701]]
CPU times: user 4.75 ms, sys: 0 ns, total: 4.75 ms
Wall time: 4.31 ms


In [53]:
[f'{i}: {sentences[i]}' for i in I[0]]

['9601: Two dogs run through the brush.',
 '6493: Two dogs are running on the grass in front of the trees.',
 '1379: two dogs running in grass',
 '7701: A dog is running along the grass.']

In [54]:
index.make_direct_map()